In [39]:
import pandas as pd
from io import StringIO
import os
import numpy as np
import matplotlib.pyplot as plt

In [40]:
def read_trace(trace_dir, trace_file, worker_id=None):
    trace_filepath = os.path.join(trace_dir, trace_file)
    
    with open(trace_filepath,"r") as rf:
        new_trace = StringIO()
        line = rf.readline()
        while line:
            line_list = line.split()
            if line_list[0] == "[trace]" or line_list[0] == "[PROF]":
                new_trace.write(line)
            line = rf.readline()
       
    new_trace.seek(0)
    df = pd.read_csv(new_trace, sep=" ", names=["InfoType", "EventType", "EventState", "ID", "Time"])
    del df["InfoType"]
    
    if worker_id is not None:
        df.insert(0, "WorkerID", worker_id)
        
    new_trace.close()
    
    return df

In [41]:
recv_ratio_list = []
message_size_list = []
avg_recv_ratio_list = []
avg_duration_list = []
# dir_id = "0901"
# p_size = "5000000"
# trace_dir = "/home/posheng/billy_ws/fl_025/paslab/trace/async2_fl_sagg_3rm_mask_nossl/" + dir_id + "/send_vs_recv"

dir_id = "async2_fl_bw2_3rm"
bw = "100mbps"
p_size = "4000000"
trace_dir = "/home/posheng/billy_ws/fl_025/paslab/trace/" + dir_id + "/" + bw + "/send_vs_recv"
coord_trace_file = "coord_" + p_size + ".txt"

coord_df = read_trace(trace_dir, coord_trace_file)
coord_df = coord_df[4:].reset_index()
del coord_df["index"]

avg_duration = coord_df[coord_df["EventType"] == "ModelSend"][1:]["Time"].mean()
avg_duration_list.append(avg_duration)

start_indices = list(coord_df[coord_df["EventType"] == "MessageSize"].index)
for start_index in start_indices:
    message_size = int(coord_df.loc[start_index]["EventState"])
    send_time = coord_df.loc[start_index+2]["Time"] - coord_df.loc[start_index+1]["Time"]
    recv_time = coord_df.loc[start_index+3]["Time"] - coord_df.loc[start_index+2]["Time"]
    recv_ratio = recv_time / (send_time + recv_time)
    recv_ratio_list.append(recv_ratio)

message_size_list.append(message_size)
avg_recv_ratio = sum(recv_ratio_list) / len(recv_ratio_list)
avg_recv_ratio_list.append(avg_recv_ratio)

In [42]:
avg_recv_ratio_list

[0.6338136808467709]

In [43]:
message_size_list

[48007125]

In [44]:
message_size_list

[48007125]

In [38]:
11544930 * 4

46179720